(echopop-example-workflow)=

# **Example overview of the `echopop` dataflow**

## **`Survey`-class initialization**

Import the latest version of `echopop`.

In [36]:
import pprint

from echopop.survey import Survey

Initialize the `Survey` object by loading in input data (`Survey.input`) and configuration settings (`Survey.config`). The former reads in data from all the defined input files contained within the `./config_files/survey_year_2019_config.yml` configuration file. The latter reads in various arguments as well as the file paths that point to the input files. 

In [38]:
survey = Survey( init_config_path = "C:/Users/Brandyn/Documents/GitHub/echopop/config_files/initialization_config.yml" ,
                 survey_year_config_path = "C:/Users/Brandyn/Documents/GitHub/echopop/config_files/survey_year_2019_config.yml" )

Not only are all the necessary acoustic, biological, kriging, and stratification data imported and contained with `survey`, but they can also be parsed in a relatively straightforward manner. There are five `Survey`-class attributes to be aware of: 
* `Survey.meta`: this is currently undeveloped, but this is where necessary information such as the date the object was created and general data workflow/provenance would be collected.
* `Survey.config`: this stores the background configuration settings. 
* `Survey.input`: this contains the imported acoustic, biological, kriging, and stratification data. This can be further investigated via the various nested dictionaries that correspond to specific types of dataset. 
* `Survey.analysis`: this is the working directory that contains relevant intermediate data products and calculations that may be of interest to the user and/or are required for later calculations. 
* `Survey.results`: this stores the overall results each analysis. 
  
## **Initial data processing**

**`Survey.meta`**

As previously mentioned, `Survey.meta` is undeveloped, but the `provenance` key will be iteratively updated with the performed analyses. Additional metadata can also be appended to this attribute.

In [39]:
pprint.pprint( survey.meta )

{'date': '2024-05-28 07:13:38', 'provenance': {}}


**`Survey.config`**

This attribute contains a variety of nested dictionaries that help to organize the entries in an intentional format that ideally minimizes ambiguity on how to access the associated values. Accessible dictionaries can be listed via `survey.config.keys()`:

In [40]:
survey.config.keys( )

dict_keys(['stratified_survey_mean_parameters', 'TS_length_regression_parameters', 'geospatial', 'survey_year', 'species', 'CAN_haul_offset', 'data_root_dir', 'biological', 'stratification', 'NASC', 'kriging'])

The overall dictionary structure of `self.config` can also be accessed. Although not required for printing out the values in this attribute, the `pprint` library is helpful for formatting nested dictionaries into a legible format in both the console and interactive notebooks. 

In [41]:
pprint.pprint( survey.config )

{'CAN_haul_offset': 200,
 'NASC': {'all_ages': {'filename': 'Exports/US_CAN_detailsa_2019_table1y+_ALL_final '
                                   '- updated.xlsx',
                       'sheetname': 'Sheet1'},
          'no_age1': {'filename': 'Exports/US_CAN_detailsa_2019_table2y+_ALL_final '
                                  '- updated.xlsx',
                      'sheetname': 'Sheet1'}},
 'TS_length_regression_parameters': {'pacific_hake': {'TS_L_intercept': -68.0,
                                                      'TS_L_slope': 20.0,
                                                      'length_units': 'cm',
                                                      'number_code': 22500}},
 'biological': {'catch': {'CAN': {'filename': 'Biological/CAN/2019_biodata_catch_CAN.xlsx',
                                  'sheetname': 'biodata_catch_CAN'},
                          'US': {'filename': 'Biological/US/2019_biodata_catch.xlsx',
                                 'sheetname': 'biod

**`Survey.input`**

Similar to `Survey.config`, the input data are grouped into various nested dictionaries. Data contained within the `Survey.input` attribute are specifically stored in four general nested dictionaries: `acoustics`, `biology`, `spatial`, and `statistics`. 

In [42]:
survey.input.keys()

dict_keys(['acoustics', 'biology', 'spatial', 'statistics'])

This results in the following branched data structure for `Survey.input`:
* `acoustics`
  * `nasc_df`: acoustic trawl data (all age and age-2+ NASC)
* `biology`
  * `catch_df`: unaged haul weight totals
  * `distributions`
    * `age_bins_df`: age distribution histogram bins
    * `length_bins_df`: length distribution histogram bins
  * `haul_to_transect_df`: haul-to-transect key that links haul numbers to their respective transects
  * `length_df`: unaged length measurements
  * `specimen_df`: aged length and weight measurements
* `spatial`
  * `strata_df`: the `KS` stratum definitions and fraction of hake for each haul
  * `geo_strata_df`: latitudinal limits of the `KS` strata
  * `inpfc_strata_df`: the `INPFC` stratum definitions and their respective latitudinal limits
* `statistics`
  * `kriging`
    * `mesh_df`: kriging mesh
    * `isobath_200m_df`: 200 m isobath coordinates
    * `model_config`: dictionary comprising all required arguments for the kriging analysis
  * `variogram`
    * `model_config`: dictionary comprising all required arguments for the variogram analysis

### **`Survey.transect_analysis(...)`**

<div class="alert alert-block alert-success">
<b>TIP:</b> Hover your cursor over the various functions included in the code blocks below to get additional type hints and context for usage
</div>

The method `Survey.transect_analysis(...)` populates various analysis variables (`Survey.analysis`) and results (`Survey.results`). This class-method currently takes four user arguments:

* `species_id (integer, list)`: the species number code(s) (default: `22500`)
* `exclude_age1 (boolean)`: whether age-1 fish should be excluded from the analysis (default: `True`)
* `stratum (string)`: the stratum used for the various acoustic and biological calculations (default: `'ks'`)
* `verbose (boolean)`: dialogue messages will appear in the console including a summary report of the results when this is set to `True` (default: `True`) 
  
This is the primary biological data processing workhorse that is further used for later analyses, such as computing the number and weight proportions across all animals.

In [43]:
survey.transect_analysis( species_id = 22500 , exclude_age1 = True , stratum = 'ks' , verbose = True )


    --------------------------------
    TRANSECT RESULTS
    --------------------------------
    | Variable: Biomass (kmt)
    | Age-1 fish excluded: True
    | Stratum definition: KS
    --------------------------------
    GENERAL RESULTS
    --------------------------------
    Total biomass: 1651.1 kmt
        Age-1: 7.9 kmt
        Age-2+: 1643.2 kmt
    Total female biomass: 832.2 kmt
        Age-1: 4.0 kmt
        Age-2+: 828.2 kmt
    Total male biomass: 818.5 kmt
        Age-1: 3.9 kmt
        Age-2+: 814.6 kmt
    Total unsexed biomass: 0.4 kmt
    Total mixed biomass: 36.8 kmt
    --------------------------------


A variety of intermediate data products are stored in `Survey.analysis` under currently four nested dictionaries: 
* `kriging`: intermediate results specific to the kriging analysis (`Survey.kriging_analysis(...)`)
* `settings`: this provides a full recording of user-inputs and other variable definitions used for each analysis to improve replicability
* `stratified`: intermediate results specific to the stratified sampling analysis (`Survey.stratified_analysis(...)`)
* `transect`: intermediate results specific to the transect analysis (`Survey.transect_analysis(...)`)

In [44]:
survey.analysis.keys( )

dict_keys(['transect', 'settings', 'stratified'])

Since `Survey.transect_analysis(...)` was ran, the specific arguments used for the analysis can be directly accessed via:

In [45]:
pprint.pprint( survey.analysis[ 'settings' ][ 'transect' ] )

{'exclude_age1': True,
 'species_id': 22500,
 'stratum': 'ks',
 'stratum_name': 'stratum_num'}


The intermediate data products can be similarly accessed under the `transect` dictionary within `Survey.analysis`: 

In [46]:
survey.analysis[ 'transect' ].keys()

dict_keys(['acoustics', 'biology', 'coordinates'])

The results from each analysis are then stored within the `Survey.results` attribute:

In [47]:
survey.results.keys()

dict_keys(['transect', 'stratified', 'kriging'])

So we can generally glean all results recorded within `Survey.results` and also access those specific to `Survey.transect_analysis(...)` within `transect`:

In [48]:
pprint.pprint( survey.results )

{'kriging': {},
 'stratified': {},
 'transect': {'biomass_summary_df':        sex  biomass_adult  biomass_age1   biomass_all
0      all   1.643215e+09  7.869992e+06  1.651085e+09
1   female   8.282280e+08  3.950822e+06  8.321788e+08
2     male   8.146257e+08  3.919170e+06  8.185449e+08
3  unsexed   3.609296e+05  0.000000e+00  3.609296e+05
4    mixed   3.680784e+07  0.000000e+00  3.680784e+07}}


In [49]:
survey.results[ 'transect' ]

{'biomass_summary_df':        sex  biomass_adult  biomass_age1   biomass_all
 0      all   1.643215e+09  7.869992e+06  1.651085e+09
 1   female   8.282280e+08  3.950822e+06  8.321788e+08
 2     male   8.146257e+08  3.919170e+06  8.185449e+08
 3  unsexed   3.609296e+05  0.000000e+00  3.609296e+05
 4    mixed   3.680784e+07  0.000000e+00  3.680784e+07}

## **`Survey.stratified_analysis(...)`**

`Survey.stratified_analysis(...)` computes various stratified statistics, including the coefficient of variation (*CV*) estimates using the Jolly and Hampton (1990) stratified sampling method. There are a variety of arguments used for this function: 
* `dataset ('transect', 'kriging')`: data input selection (default: `'transect'`). This will use either the results of `Survey.transect_analysis(...)` or `Survey.kriging_analysis(...)`
* `stratum ('ks','inpfc')`: the stratum used for the various acoustic and biological calculations (default: `'inpfc'`)
* `variable( 'abundance' , 'biomass' , 'nasc')`: the data variable that will be used for the stratified resampling analysis (default: `'biomass'`)
* `verbose (boolean)`: dialogue messages will appear in the console including a summary report of the results when this is set to `True` (default: `True`)

There are also analysis-specific optional arguments that are used depending on how `dataset` is defined:

* `mesh_transect_per_latitude (integer)`: the number of virtual transects per degree latitude when `dataset = 'kriging'`
* `transect_sample`: the resampling proportion used to resample transects within each stratum without replacement (default: inherits value from `Survey.config['stratified_survey_mean_parameters']`)
* `transect_replicates`: the number of resampling iterations that will be run (default: inherits value from `Survey.config['stratified_survey_mean_parameters']`)


In [50]:
survey.stratified_analysis( dataset = 'transect' , stratum = 'inpfc' , variable = 'biomass' , verbose = True )


    --------------------------------
    STRATIFIED RESULTS (TRANSECT)
    --------------------------------
    | Stratified variable: Biomass (kmt)
    | Number of transects: 113
    | Total strata area coverage: 53509.0 nmi^2
    | Age-1 fish excluded: True
    | Stratum definition: INPFC
    | Bootstrap replicates: 10000 samples
    | Resampling proportion: 0.75
    --------------------------------
    STRATUM-SPECIFIC ESTIMATES
    --------------------------------
    _coming soon_
    --------------------------------
    GENERAL RESULTS
    --------------------------------
    CV: 0.1329 [0.1238, 0.1419; 95% CI]
    Total (across sub-sampled transects): 1227.6 kmt [1074.8, 1380.4; 95% CI]
          (back-transformed): 1636.8 kmt [1433.1, 1840.5; 95% CI]
    Mean (across sub-sampled transects):        203.4 kmt [172.1, 234.6; 95% CI]
    --------------------------------


<div class="alert alert-block alert-warning">
<b>NOTE:</b> You cannot run `Survey.stratified_analysis( dataset = 'kriging' , ... )` unless you have already computed the kriging results via `Survey.kriging_analysis(...)`.
</div>

Depending on how `dataset` is parameterized, the intermediate and final results are stored within a sub-dictionary so the outputs from both `dataset = 'transect'` and `dataset = 'kriging'` can be compared. For `Survey.analysis`, these are separated immediately below the top-level dictionary: 

In [51]:
survey.analysis[ 'stratified' ].keys( )

dict_keys(['transect'])

Here the resampled distributions of multiple statistics can be directly accessed for additional uncertainty analyses and visualizing the underlying statistical distributions: 

In [52]:
survey.analysis[ 'stratified' ][ 'transect' ].keys()

dict_keys(['stratified_replicates_df'])

In [53]:
survey.analysis[ 'stratified' ][ 'transect' ][ 'stratified_replicates_df' ]

,realization,mean_standardized,standard_deviation,variance,cv,mean,total
0,1,8.421319e+11,1.077189e+11,1.160337e+22,0.127912,2.099707e+08,1.194333e+09
1,2,9.374169e+11,1.218862e+11,1.485625e+22,0.130023,2.338089e+08,1.394089e+09
2,3,7.952674e+11,1.029277e+11,1.059411e+22,0.129425,1.929758e+08,1.131455e+09
3,4,8.666584e+11,1.147664e+11,1.317133e+22,0.132424,2.092199e+08,1.273637e+09
4,5,8.381991e+11,1.133317e+11,1.284406e+22,0.135209,2.022384e+08,1.222494e+09
...,...,...,...,...,...,...,...
9995,9996,8.784094e+11,1.180293e+11,1.393092e+22,0.134367,2.137757e+08,1.296847e+09
9996,9997,8.133567e+11,1.081531e+11,1.169708e+22,0.132971,1.993702e+08,1.201448e+09
9997,9998,8.169507e+11,1.044826e+11,1.091661e+22,0.127893,2.020326e+08,1.194972e+09
9998,9999,7.087371e+11,1.018084e+11,1.036495e+22,0.143648,1.780005e+08,1.139261e+09


The final results stored within `Survey.results` are formatted in an identical way:

In [54]:
survey.results[ 'stratified' ].keys( )

dict_keys(['transect'])

In [55]:
survey.results[ 'stratified' ][ 'transect' ].keys()

dict_keys(['variable', 'ci_percentile', 'num_transects', 'total_area', 'mean', 'variance', 'cv', 'total'])

In [56]:
pprint.pprint( survey.results[ 'stratified' ][ 'transect' ])

{'ci_percentile': 0.95,
 'cv': {'confidence_interval': array([0.1238229 , 0.14190188]),
        'estimate': 0.13286238737496395},
 'mean': {'unweighted_confidence_interval': array([1.72072437e+08, 2.34630809e+08]),
          'unweighted_estimate': 203351623.02741227,
          'weighted_confidence_interval': array([7.04140693e+11, 9.60798852e+11]),
          'weighted_estimate': 832469772839.8756},
 'num_transects': 113,
 'total': {'confidence_interval': array([1.07480518e+09, 1.38038733e+09]),
           'estimate': 1227596254.6756964},
 'total_area': 53508.99736238332,
 'variable': 'biomass',
 'variance': {'confidence_interval': array([9.14291115e+21, 1.53506205e+22]),
              'estimate': 1.2246765839809253e+22}}


## **`Survey.kriging_analysis(...)`**

`Survey.kriging_analysis(...)` computes the kriged estimates for the target variable via ordinary kriging with an adaptive search radius. The arguments to `Survey.kriging_analysis(...)` include:
* `coordinate_transform (boolean)`: when `True`, the transect and mesh longitude/latitude coordinates are transformed to a standardized format as x/y (default: `True`)
* `crop_method ('interpolation', 'convex_hull')`: when `extrapolate = False`, this determines the method used for cropping the kriging mesh. Setting `crop_method = 'interpolation'` (*default*) resamples the latitudinal resolution of the mesh grid and interpolates over the extent of the eastern and western endpoints of each transect line. This is conducted in a piece-wise fashion to account for the island of Haida Gwaii. Setting `crop_method = 'convex_hull'` uses a polygon-based approach for cropping the mesh grid based on the survey extent.
* `extrapolate(boolean)`: when `True`, the entire kriging mesh is used. Otherwise, different methods are used to crop the kriging mesh to limit extrapolation beyond the extent of the survey transects. 
* `stratum ('ks','inpfc')`: the stratum used for mapping the defined kriged `variable` (default: `'ks'`) 
* `variable(string)`: the data variable that will be used for the kriging analysis (default: `'biomass_density'`)
* `verbose (boolean)`: dialogue messages will appear in the console including a summary report of the results when this is set to `True` (default: `True`)

There are also analysis-specific optional arguments that are used depending on how `crop_method` is defined:
* When `crop_method = 'interpolation'`:
  * `latitude_resolution (float)`: the updated latitudinal resolution (**in nmi**) used for interpolation
* When `crop_method = 'convex_hull'`:
  * `mesh_buffer_distance`: this is a dilation factor (**in nmi**) that expands/buffers the extent of the polygon defining the survey extent (default: `1.25`)
  * `num_nearest_transects`: this defines the number of nearest neighboring transects used for generating smaller polygons that are then constructed into the survey-wide polygon

Lastly, there are additional arguments that are optional since they are otherwise inherited from various parts of the `Survey` object: 
* `kriging_parameters (dictionary)`: a dictionary containing various kriging parameter variables and arguments
* `projection (string)`: an EPSG string that defines the mapping projection
* `variogram_parameters (dictionary)`: a dictionary containing various variogram parameter variables and arguments

In [57]:
survey.kriging_analysis( bearing_tolerance = 15.0 , coordinate_transform = True , crop_method = 'interpolation' , extrapolate = False , latitude_resolution = 1.25 , stratum = 'ks' , variable = 'biomass_density' , verbose = True )

Longitude and latitude coordinates (WGS84) converted to standardized coordinates (x and y).
Extrapolation applied to kriging mesh points (81 of 9463):
            * 77 points had 0 valid range estimates without extrapolation
            * 4 points had at least 1 valid point but fewer than 3 valid neighbors
Imputed apportioned unaged male biomass at length bins:
(17.0, 19.0], (59.0, 61.0], (61.0, 63.0], (63.0, 65.0], (65.0, 67.0], (67.0, 69.0], (69.0, 71.0], (71.0, 73.0], (73.0, 75.0], (75.0, 77.0]
Imputed apportioned unaged female biomass at length bins:
(17.0, 19.0], (73.0, 75.0], (75.0, 77.0]

    --------------------------------
    KRIGING RESULTS (MESH)
    --------------------------------
    | Kriged variable: Biomass density (kg/nmi^2)
    | Age-1 fish excluded: True
    | Stratum definition: KS
    | Mesh extrapolation: False
    --- Mesh cropping method: Interpolation
    | Mesh and transect coordinate standardization: True
    --------------------------------
    GENERAL RES

There are then various results stored within `Survey.results[ 'kriging' ]`:

In [58]:
survey.results[ 'kriging' ].keys()

dict_keys(['variable', 'survey_mean', 'survey_estimate', 'survey_cv', 'mesh_results_df', 'tables'])

Some of these values are single values:

In [59]:
pprint.pprint( [survey.results['kriging'].get(key) for key in ['variable' , 'survey_mean' , 'survey_estimate' , 'survey_cv'] ] )

['biomass_density', 27812.72904530149, 1644729100.704989, 0.02827148127046275]


The meshed results can also be retrieved:

In [60]:
survey.results[ 'kriging' ][ 'mesh_results_df' ]

,latitude,longitude,area,kriged_mean,kriged_variance,sample_variance,sample_cv,biomass,stratum_num
1,49.057959,-126.024127,6.250000,0.000000,0.031410,NaN,0.008405,0.000000,7
2,49.016196,-126.024110,6.250000,0.000000,0.278348,NaN,0.025021,0.000000,7
3,48.974438,-126.024093,6.250000,0.000000,0.584354,NaN,0.036254,0.000000,7
4,48.932686,-126.024076,6.250000,48215.463859,0.739334,1.303223,0.040779,301346.649118,7
5,48.890939,-126.024060,6.250000,0.000000,0.783366,NaN,0.041975,0.000000,8
...,...,...,...,...,...,...,...,...,...
19804,52.895008,-132.337719,0.011343,0.000000,0.947455,NaN,0.046163,0.000000,1
19806,52.813140,-132.260812,0.009924,0.000000,0.539868,NaN,0.034846,0.000000,1
19814,38.025533,-123.013372,0.006006,0.000000,0.346581,NaN,0.027920,0.000000,5
19830,35.646423,-121.257388,0.001815,0.000000,0.365594,NaN,0.028676,0.000000,3


The `tables` sub-dictionary includes the sum of each variable distributed over age, length, and sex (in this case, `variable = biomass_density` produces estimates of kriged `biomass` for these tables).

Biomass distributed over age, length, and sex for aged fish:

In [61]:
survey.results['kriging']['tables'][ 'aged_tbl' ]

age_bin              (0.5, 1.5]  (1.5, 2.5]  (2.5, 3.5]  (3.5, 4.5]  \
sex    length_bin                                                     
female (1.0, 3.0]           0.0         0.0         0.0         0.0   
       (3.0, 5.0]           0.0         0.0         0.0         0.0   
       (5.0, 7.0]           0.0         0.0         0.0         0.0   
       (7.0, 9.0]           0.0         0.0         0.0         0.0   
       (9.0, 11.0]          0.0         0.0         0.0         0.0   
...                         ...         ...         ...         ...   
male   (71.0, 73.0]         0.0         0.0         0.0         0.0   
       (73.0, 75.0]         0.0         0.0         0.0         0.0   
       (75.0, 77.0]         0.0         0.0         0.0         0.0   
       (77.0, 79.0]         0.0         0.0         0.0         0.0   
       (79.0, 81.0]         0.0         0.0         0.0         0.0   

age_bin              (4.5, 5.5]  (5.5, 6.5]  (6.5, 7.5]  (7.5, 8.5]  \
sex    length_bin                                                     
female (1.0, 3.0]           0.0         0.0         0.0         0.0   
       (3.0, 5.0]           0.0         0.0         0.0         0.0   
       (5.0, 7.0]           0.0         0.0         0.0         0.0   
       (7.0, 9.0]           0.0         0.0         0.0         0.0   
       (9.0, 11.0]          0.0         0.0         0.0         0.0   
...                         ...         ...         ...         ...   
male   (71.0, 73.0]         0.0         0.0         0.0         0.0   
       (73.0, 75.0]         0.0         0.0         0.0         0.0   
       (75.0, 77.0]         0.0         0.0         0.0         0.0   
       (77.0, 79.0]         0.0         0.0         0.0         0.0   
       (79.0, 81.0]         0.0         0.0         0.0         0.0   

age_bin              (8.5, 9.5]  (9.5, 10.5]  ...  (12.5, 13.5]  (13.5, 14.5]  \
sex    length_bin                             ...                               
female (1.0, 3.0]           0.0          0.0  ...           0.0           0.0   
       (3.0, 5.0]           0.0          0.0  ...           0.0           0.0   
       (5.0, 7.0]           0.0          0.0  ...           0.0           0.0   
       (7.0, 9.0]           0.0          0.0  ...           0.0           0.0   
       (9.0, 11.0]          0.0          0.0  ...           0.0           0.0   
...                         ...          ...  ...           ...           ...   
male   (71.0, 73.0]         0.0          0.0  ...           0.0           0.0   
       (73.0, 75.0]         0.0          0.0  ...           0.0           0.0   
       (75.0, 77.0]         0.0          0.0  ...           0.0           0.0   
       (77.0, 79.0]         0.0          0.0  ...           0.0           0.0   
       (79.0, 81.0]         0.0          0.0  ...           0.0           0.0   

age_bin              (14.5, 15.5]  (15.5, 16.5]  (16.5, 17.5]  (17.5, 18.5]  \
sex    length_bin                                                             
female (1.0, 3.0]             0.0           0.0           0.0           0.0   
       (3.0, 5.0]             0.0           0.0           0.0           0.0   
       (5.0, 7.0]             0.0           0.0           0.0           0.0   
       (7.0, 9.0]             0.0           0.0           0.0           0.0   
       (9.0, 11.0]            0.0           0.0           0.0           0.0   
...                           ...           ...           ...           ...   
male   (71.0, 73.0]           0.0           0.0           0.0           0.0   
       (73.0, 75.0]           0.0           0.0           0.0           0.0   
       (75.0, 77.0]           0.0           0.0           0.0           0.0   
       (77.0, 79.0]           0.0           0.0           0.0           0.0   
       (79.0, 81.0]           0.0           0.0           0.0           0.0   

age_bin              (18.5, 19.5]  (19.5, 20.5]  (20.5, 21.5]  (21.5, 

Biomass distributed over length and sex for unaged fish:

In [65]:
survey.results['kriging']['tables']['unaged_tbl']

sex,female,male
length_bin,,
"(1.0, 3.0]",0.000000e+00,0.000000e+00
"(3.0, 5.0]",0.000000e+00,0.000000e+00
"(5.0, 7.0]",0.000000e+00,0.000000e+00
"(7.0, 9.0]",0.000000e+00,0.000000e+00
"(9.0, 11.0]",0.000000e+00,0.000000e+00
"(11.0, 13.0]",0.000000e+00,0.000000e+00
"(13.0, 15.0]",0.000000e+00,0.000000e+00
"(15.0, 17.0]",0.000000e+00,0.000000e+00
"(17.0, 19.0]",5.703125e+03,6.802037e+03


Combined biomass from both the aged and unaged fish distributed over length, age, and sex: 

In [64]:
survey.results['kriging']['tables']['overall_apportionment_df']

,age_bin,sex,length_bin,biomass_apportioned
0,"(0.5, 1.5]",all,"(1.0, 3.0]",0.0
1,"(0.5, 1.5]",female,"(1.0, 3.0]",0.0
2,"(0.5, 1.5]",male,"(1.0, 3.0]",0.0
3,"(1.5, 2.5]",all,"(1.0, 3.0]",0.0
4,"(1.5, 2.5]",female,"(1.0, 3.0]",0.0
...,...,...,...,...
2635,"(20.5, 21.5]",female,"(79.0, 81.0]",0.0
2636,"(20.5, 21.5]",male,"(79.0, 81.0]",0.0
2637,"(21.5, 22.5]",all,"(79.0, 81.0]",0.0
2638,"(21.5, 22.5]",female,"(79.0, 81.0]",0.0


Now that the kriging results are computed, they can then be used to parameterize `Survey.stratified_analysis( dataset = 'kriging' , ...)` to conduct the stratified resampling analysis: 

In [33]:
survey.stratified_analysis( 'kriging' )


    --------------------------------
    STRATIFIED RESULTS (KRIGING)
    --------------------------------
    | Stratified variable: Biomass (kmt)
    | Number of virtual transects: 102
    | Total strata area coverage: 35289.9 nmi^2
    | Age-1 fish excluded: True
    | Stratum definition: INPFC
    | Bootstrap replicates: 10000 samples
    | Resampling proportion: 0.75
    --------------------------------
    STRATUM-SPECIFIC ESTIMATES
    --------------------------------
    _coming soon_
    --------------------------------
    GENERAL RESULTS
    --------------------------------
    CV: 0.1373 [0.1301, 0.1445; 95% CI]
    Total (across sub-sampled transects): 1230.7 kmt [1098.4, 1363.0; 95% CI]
          (back-transformed): 1640.9 kmt [1464.5, 1817.3; 95% CI]
    Mean (across sub-sampled transects):        191.4 kmt [162.8, 220.0; 95% CI]
    --------------------------------


## **Other 'useful' features**

Although a summary of the results are printed in the console when `verbose = True`, it is a bit obnoxious to have to re-run the entire analysis to re-generate the same message. This is addressed via the `Survey.summary(...)` method that comprises a single input: 
* `results_name (string)`: this is the name of the analysis results that should be printed into the console. This can either be formatted as a single input name (e.g. 'transect' , 'kriging') or a nested/layered variable (e.g. 'stratified:transect') where a colon (':') is used as the delimiter that separates the two result layer names.

In [34]:
survey.summary( 'transect' )


    --------------------------------
    TRANSECT RESULTS
    --------------------------------
    | Variable: Biomass (kmt)
    | Age-1 fish excluded: True
    | Stratum definition: KS
    --------------------------------
    GENERAL RESULTS
    --------------------------------
    Total biomass: 1651.1 kmt
        Age-1: 7.9 kmt
        Age-2+: 1643.2 kmt
    Total female biomass: 832.2 kmt
        Age-1: 4.0 kmt
        Age-2+: 828.2 kmt
    Total male biomass: 818.5 kmt
        Age-1: 3.9 kmt
        Age-2+: 814.6 kmt
    Total unsexed biomass: 0.4 kmt
    Total mixed biomass: 36.8 kmt
    --------------------------------


In [35]:
survey.summary( 'stratified:transect' )


    --------------------------------
    STRATIFIED RESULTS (KRIGING)
    --------------------------------
    | Stratified variable: Biomass (kmt)
    | Number of virtual transects: 113
    | Total strata area coverage: 53509.0 nmi^2
    | Age-1 fish excluded: True
    | Stratum definition: INPFC
    | Bootstrap replicates: 10000 samples
    | Resampling proportion: 0.75
    --------------------------------
    STRATUM-SPECIFIC ESTIMATES
    --------------------------------
    _coming soon_
    --------------------------------
    GENERAL RESULTS
    --------------------------------
    CV: 0.1328 [0.1239, 0.1417; 95% CI]
    Total (across sub-sampled transects): 1227.9 kmt [1074.9, 1380.9; 95% CI]
          (back-transformed): 1637.2 kmt [1433.2, 1841.2; 95% CI]
    Mean (across sub-sampled transects):        203.5 kmt [172.2, 234.7; 95% CI]
    --------------------------------


In [36]:
survey.summary( 'stratified:kriging' )


    --------------------------------
    STRATIFIED RESULTS (KRIGING)
    --------------------------------
    | Stratified variable: Biomass (kmt)
    | Number of virtual transects: 102
    | Total strata area coverage: 35289.9 nmi^2
    | Age-1 fish excluded: True
    | Stratum definition: INPFC
    | Bootstrap replicates: 10000 samples
    | Resampling proportion: 0.75
    --------------------------------
    STRATUM-SPECIFIC ESTIMATES
    --------------------------------
    _coming soon_
    --------------------------------
    GENERAL RESULTS
    --------------------------------
    CV: 0.1373 [0.1301, 0.1445; 95% CI]
    Total (across sub-sampled transects): 1230.7 kmt [1098.4, 1363.0; 95% CI]
          (back-transformed): 1640.9 kmt [1464.5, 1817.3; 95% CI]
    Mean (across sub-sampled transects):        191.4 kmt [162.8, 220.0; 95% CI]
    --------------------------------


In [37]:
survey.summary( 'kriging' )


    --------------------------------
    KRIGING RESULTS (MESH)
    --------------------------------
    | Kriged variable: Biomass density (kg/nmi^2)
    | Age-1 fish excluded: True
    | Stratum definition: KS
    | Mesh extrapolation: False
    --- Mesh cropping method: Interpolation
    | Mesh and transect coordinate standardization: True
    --------------------------------
    GENERAL RESULTS
    --------------------------------
    Mean biomass density: 27812.73 kg/nmi^2
    Total survey biomass estimate: 1644.73 kmt
    Mean mesh sample CV: 0.0255
    Overall survey CV: 0.0283
    Total area coverage: 58186.9 nmi^2
   --------------------------------
